#**PROYECTO KAGGLE - Pruebas Saber Pro Colombia**
## **Solución Alternativa**

Modelo Predictivo a partir de la tabla de datos train.csv sobre el rendimiento de los estudiantes en las pruebas Saber Pro.

Hecho por:
- Juan Manuel Areiza Ospina - C.C. 1018226898
- Samuel Puerta Patiño - C.C. 1023624795
- Brayan Stiven Gómez Villa - C.C 1018224235

La solución utiliza XGBoost, un algoritmo robusto y eficiente para datos tabulares. El proceso incluye:

1. **Carga de Datos:** Importación de los conjuntos de entrenamiento y prueba.

2. **Ingeniería de Características:** Transformación profunda de variables categóricas (estrato, educación) y creación de nuevos índices sintéticos para capturar mejor el contexto socioeconómico del estudiante.

3. **Codificación:** Uso de OrdinalEncoder y LabelEncoder para preparar los datos para el modelo.

4. **Modelado:** Entrenamiento de un clasificador XGBoost con hiperparámetros optimizados para evitar el sobreajuste (overfitting).

## **Carga de Datos**

In [1]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 362MB/s]


In [2]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


## **Modelo**

In [3]:
# INSTALACIÓN DE XGBOOST
!pip install xgboost

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer


#CARGA DE DATOS
print("Cargando datos")
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
submission_example = pd.read_csv('submission_example.csv')

# PROCESAMIENTO
def procesar_datos(df):
    df = df.copy()

    # Limpieza básica
    # Eliminar duplicados  o columnas vacías
    cols_drop = ['F_TIENEINTERNET.1'] # Duplicada
    df = df.drop(columns=[c for c in cols_drop if c in df.columns], errors='ignore')

    # Mapeos de Ordinales
    # Mapa de Estrato
    mapa_estrato = {
        'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3,
        'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6, 'Sin Estrato': 0
    }
    df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(mapa_estrato).fillna(0)

    # Mapa de Educación (Aproximado por nivel)
    mapa_edu = {
        'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
        'Postgrado': 9
    }
    # Aplicar a padre y madre
    df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].map(mapa_edu).fillna(-1)
    df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].map(mapa_edu).fillna(-1)

    # Mapa Si/No
    mapa_si_no = {'Si': 1, 'No': 0}
    cols_binarias = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR', 'E_PAGOMATRICULAPROPIO']
    for col in cols_binarias:
        if col in df.columns:
            df[col] = df[col].map(mapa_si_no).fillna(0)

    # Creación de nuevas variables

    # Índice de equipamiento tecnológico/hogar (suma de binarias)
    df['SCORE_HOGAR'] = df['F_TIENEINTERNET'] + df['F_TIENELAVADORA'] + df['F_TIENEAUTOMOVIL'] + df['F_TIENECOMPUTADOR']

    # Promedio de educación de los padres
    df['EDU_PADRES_AVG'] = (df['F_EDUCACIONPADRE'] + df['F_EDUCACIONMADRE']) / 2

    # Diferencia entre indicadores (si existen)
    if 'INDICADOR_1' in df.columns and 'INDICADOR_2' in df.columns:
        df['DELTA_IND'] = df['INDICADOR_1'] - df['INDICADOR_2']

    return df

print("Procesando Features...")
X = procesar_datos(df_train.drop(columns=['RENDIMIENTO_GLOBAL', 'ID']))
X_test_final = procesar_datos(df_test.drop(columns=['ID']))
y = df_train['RENDIMIENTO_GLOBAL']


# CODIFICACIÓN FINAL

# Codificar variables categóricas restantes (Departamento, Programa, etc.)
#Se usa OrdinalEncoder para asegurar una compatibilidad simple.

cat_cols = X.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X[cat_cols] = encoder.fit_transform(X[cat_cols])
X_test_final[cat_cols] = encoder.transform(X_test_final[cat_cols])

# Codificar el TARGET (y)
le = LabelEncoder()
y_encoded = le.fit_transform(y) # Convierte alto, bajo... en 0, 1, 2, 3


#  MODELADO CON XGBOOST
# División Train/Val
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.15, random_state=42, stratify=y_encoded)

print("Entrenando XGBoost...")

model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    num_class=4,
    random_state=42,
    n_jobs=-1,
    tree_method='hist',
    early_stopping_rounds=50
)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)

# 5. EVALUACIÓN

print("\nEvaluando modelo...")
y_pred_val = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred_val))
print(classification_report(y_val, y_pred_val, target_names=le.classes_))


# 6. CREACIÓN DE SUBMISSION
print("Generando predicciones finales...")
preds_encoded = model.predict(X_test_final)
preds_labels = le.inverse_transform(preds_encoded) # Volver a texto (alto, bajo...)

submission = pd.DataFrame({
    'ID': df_test['ID'],
    'RENDIMIENTO_GLOBAL': preds_labels
})

submission.to_csv('submission_xgb.csv', index=False)
print("ARCHIVO 'submission_xgb.csv' CREADO CON ÉXITO.")

Cargando datos
Procesando Features...
Entrenando XGBoost...
[0]	validation_0-mlogloss:1.37977
[50]	validation_0-mlogloss:1.24610
[100]	validation_0-mlogloss:1.22130
[150]	validation_0-mlogloss:1.21144
[200]	validation_0-mlogloss:1.20690
[250]	validation_0-mlogloss:1.20432
[300]	validation_0-mlogloss:1.20237
[350]	validation_0-mlogloss:1.20117
[400]	validation_0-mlogloss:1.20035
[450]	validation_0-mlogloss:1.19990
[499]	validation_0-mlogloss:1.19956

Evaluando modelo...
Accuracy: 0.43477256317689533
              precision    recall  f1-score   support

        alto       0.55      0.63      0.59     26343
        bajo       0.47      0.56      0.51     25948
  medio-alto       0.33      0.27      0.30     25743
  medio-bajo       0.33      0.27      0.30     25841

    accuracy                           0.43    103875
   macro avg       0.42      0.43      0.42    103875
weighted avg       0.42      0.43      0.42    103875

Generando predicciones finales...
ARCHIVO 'submission_xgb.csv